<a href="https://colab.research.google.com/github/2000030914/2000030914/blob/main/CP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow matplotlib seaborn scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
root_path = "/content/drive/MyDrive/AnnotatedUltrasoundLiver_Dataset"

In [ ]:
print(os.listdir('/content/drive/MyDrive/AnnotatedUltrasoundLiver_Dataset'))

['Benign', 'Normal', 'Malignant']


In [ ]:
import os
import shutil

root_path = "/content/drive/MyDrive/AnnotatedUltrasoundLiver_Dataset"
clean_dataset_path = "/content/LiverDataset"

os.makedirs(clean_dataset_path, exist_ok=True)

print("Scanning dataset...\n")

for class_name in os.listdir(root_path):
    class_path = os.path.join(root_path, class_name)

    if os.path.isdir(class_path):
        image_folder = os.path.join(class_path, "image")

        if os.path.exists(image_folder):
            print(f"Found class: {class_name}")

            new_class_path = os.path.join(clean_dataset_path, class_name)
            os.makedirs(new_class_path, exist_ok=True)

            for img in os.listdir(image_folder):
                if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                    src = os.path.join(image_folder, img)
                    dst = os.path.join(new_class_path, img)
                    shutil.copy(src, dst)

print("\nDataset cleaned successfully!")

Scanning dataset...

Found class: Benign
Found class: Normal
Found class: Malignant

Dataset cleaned successfully!


In [ ]:
for cls in os.listdir("/content/LiverDataset"):
    print(cls, "->", len(os.listdir(os.path.join("/content/LiverDataset", cls))), "images")

Benign -> 200 images
Normal -> 100 images
Malignant -> 435 images


In [ ]:
img_size = 224
batch_size = 16

datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2,rotation_range=15,zoom_range=0.1,horizontal_flip=True)
train_generator = datagen.flow_from_directory(clean_dataset_path,target_size=(img_size, img_size),batch_size=batch_size,class_mode='categorical',subset='training',shuffle=True)
val_generator = datagen.flow_from_directory(clean_dataset_path,target_size=(img_size, img_size),batch_size=batch_size,class_mode='categorical',subset='validation',shuffle=False)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
